In [6]:
# 주요 라이브러리 import
import warnings
warnings.filterwarnings(action='ignore')
import time
from IPython.display import Image
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import Ridge, Lasso, ElasticNet

# 디렉토리 및 파일 이름에 맞추어 변경
font_location = 'C:/Windows/Fonts/malgun.ttf'

font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

In [139]:
df=pd.read_csv('2018 국민여행조사_국내여행.csv')

In [140]:
df['D_TRA1_SYEAR'].value_counts()

2018.0    14296
2017.0        7
Name: D_TRA1_SYEAR, dtype: int64

In [141]:
print(df['D_TRA1_SDAY'].isna().sum(), df['D_TRA1_COST'].isna().sum(), 
df['A1'].isnull().sum(), df['A1'].notnull().sum(), df['D_TRA1_CASE'].notnull().sum())

9851 9851 12306 11848 14303


In [142]:
# 여행 유형 미체크 항목을 0으로 채움
df['D_TRA1_CASE'].fillna(0, inplace=True)
df['D_TRA2_CASE'].fillna(0, inplace=True)
df['D_TRA3_CASE'].fillna(0, inplace=True)
df['D_TRA4_CASE'].fillna(0, inplace=True)

In [143]:
df['D_TRA1_CASE'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
24149    False
24150    False
24151    False
24152    False
24153    False
Name: D_TRA1_CASE, Length: 24154, dtype: bool

In [144]:
# 여행 유형이 "2.국내여행"인 레코드만 추출
df=df[(df['D_TRA1_CASE']==2) | (df['D_TRA1_CASE']==0)]
df=df[(df['D_TRA2_CASE']==2) | (df['D_TRA2_CASE']==0)]
df=df[(df['D_TRA3_CASE']==2) | (df['D_TRA3_CASE']==0)]
df=df[(df['D_TRA4_CASE']==2) | (df['D_TRA4_CASE']==0)]

In [145]:
df.groupby('D_TRA1_CASE').count()

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_SYEAR,D_TRA1_SMONTH,D_TRA1_SDAY,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
D_TRA1_CASE,,,,,,,,,,,,,,,,,,,,,
0.0,9778,9778,9778,9778,756,756,756,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,10813,10813,10813,10813,936,936,936,10813,10813,10813,...,0,0,0,0,0,0,0,0,0,0


In [146]:
# 여행 유형이 "2.국내여행"인 레코드만 추출
# df=df[df['D_TRA1_CASE']==2 | df['D_TRA2_CASE']==2]
# df=df[(df['D_TRA2_CASE']==2)]
# df=df[df['D_TRA3_CASE']==2]
# df=df[df['D_TRA4_CASE']==2]

In [147]:
## 추천도, 재방문의사 보통이상만 추출
df = df[(df['A12']>=3) & (df['A13']>=3)]

In [148]:
## 1차 필요컬럼
columns_list = [
#   ============================ 재방문 의도, 타인추천 의도
    'A13',
    'A14',
#   ============================ 여행지에서의 활동
    'A3_1',
    'A3_2',
    'A3_3',
    'A3_4',
    'A3_5',
    'A3_6',
    'A3_7',
    'A3_8',
    'A3_9',
    'A3_10',
    'A3_11',
    'A3_12',
    'A3_12',
    'A3_13',
    'A3_14',
    'A3_15',
    'A3_16',
    'A3_17',
    'A3_18',
    'A3_19',
    'A3_20',
    'A3_21',
#   ============================ 여행 시작 날짜, 종료 날짜, 방문 지역, 각 방문 지역의 시작, 종료 날짜
    'D_TRA1_SYEAR', 
    'D_TRA1_SMONTH',
    'D_TRA1_SDAY',
    'D_TRA1_EYEAR',
    'D_TRA1_EMONTH',
    'D_TRA1_EDAY',
    
    'D_TRA1_1_CITY1',
    'D_TRA1_1_TOWN1',
    'D_TRA1_1_CITY2',
    'D_TRA1_1_TOWN2',
    'D_TRA1_1_CITY3',
    'D_TRA1_1_TOWN3',
    'D_TRA1_1_SYEAR',
    'D_TRA1_1_SMONTH',
    'D_TRA1_1_SDAY',
    'D_TRA1_1_EYEAR',
    'D_TRA1_1_EMONTH',
    'D_TRA1_1_EDAY',
    'D_TRA1_2_CITY1',
    'D_TRA1_2_TOWN1',
    'D_TRA1_2_SYEAR',
    'D_TRA1_2_SMONTH',
    'D_TRA1_2_SDAY',
    'D_TRA1_2_EYEAR',
    'D_TRA1_2_EMONTH',
    'D_TRA1_2_EDAY',
#   ============================  
    'D_TRA2_SYEAR', 
    'D_TRA2_SMONTH',
    'D_TRA2_SDAY',
    'D_TRA2_EYEAR',
    'D_TRA2_EMONTH',
    'D_TRA2_EDAY',
    
    'D_TRA2_1_CITY1',
    'D_TRA2_1_TOWN1',
    'D_TRA2_1_CITY2',
    'D_TRA2_1_TOWN2',
    'D_TRA2_1_CITY3',
    'D_TRA2_1_TOWN3',
    'D_TRA2_1_SYEAR',
    'D_TRA2_1_SMONTH',
    'D_TRA2_1_SDAY',
    'D_TRA2_1_EYEAR',
    'D_TRA2_1_EMONTH',
    'D_TRA2_1_EDAY',
    'D_TRA2_2_CITY1',
    'D_TRA2_2_TOWN1',
    'D_TRA2_2_SYEAR',
    'D_TRA2_2_SMONTH',
    'D_TRA2_2_SDAY',
    'D_TRA2_2_EYEAR',
    'D_TRA2_2_EMONTH',
    'D_TRA2_2_EDAY',
#   ============================    
    'D_TRA3_SYEAR', 
    'D_TRA3_SMONTH',
    'D_TRA3_SDAY',
    'D_TRA3_EYEAR',
    'D_TRA3_EMONTH',
    'D_TRA3_EDAY',
    
    'D_TRA3_1_CITY1',
    'D_TRA3_1_TOWN1',
    'D_TRA3_1_CITY2',
    'D_TRA3_1_TOWN2',
    'D_TRA3_1_CITY3',
    'D_TRA3_1_TOWN3',
    'D_TRA3_1_SYEAR',
    'D_TRA3_1_SMONTH',
    'D_TRA3_1_SDAY',
    'D_TRA3_1_EYEAR',
    'D_TRA3_1_EMONTH',
    'D_TRA3_1_EDAY',
    'D_TRA3_2_CITY1',
    'D_TRA3_2_TOWN1',
    'D_TRA3_2_SYEAR',
    'D_TRA3_2_SMONTH',
    'D_TRA3_2_SDAY',
    'D_TRA3_2_EYEAR',
    'D_TRA3_2_EMONTH',
    'D_TRA3_2_EDAY',
#   ============================    
    'D_TRA4_SYEAR', 
    'D_TRA4_SMONTH',
    'D_TRA4_SDAY',
    'D_TRA4_EYEAR',
    'D_TRA4_EMONTH',
    'D_TRA4_EDAY',
    
    'D_TRA4_1_CITY1',
    'D_TRA4_1_TOWN1',
    'D_TRA4_1_CITY2',
    'D_TRA4_1_TOWN2',
    'D_TRA4_1_CITY3',
    'D_TRA4_1_TOWN3',
    'D_TRA4_1_SYEAR',
    'D_TRA4_1_SMONTH',
    'D_TRA4_1_SDAY',
    'D_TRA4_1_EYEAR',
    'D_TRA4_1_EMONTH',
    'D_TRA4_1_EDAY',
    'D_TRA4_2_CITY1',
    'D_TRA4_2_TOWN1',
    'D_TRA4_2_SYEAR',
    'D_TRA4_2_SMONTH',
    'D_TRA4_2_SDAY',
    'D_TRA4_2_EYEAR',
    'D_TRA4_2_EMONTH',
    'D_TRA4_2_EDAY',
#   ============================  사전예약사항(숙소 등)
    'A8_1',
    'A8_2',
    'A8_3',
    'A8_4',
    'A8_5',
    'A8_6',
    'A8_7',
    'A8_10',
    'A8_11',
#   ============================  주요 이동 교통수단 1~3순위
    'A5_1',
    'A5_2',
    'A5_3',
#   ============================  여행지 선택 이유 1~3순위
    'A2_1',
    'A2_2',
    'A2_3',
#   ============================  동반자 유형
    'A7B_1',
    'A7B_2',
    'A7B_3',
    'A7B_4',
    'A7B_5',
    'A7B_6',
    'A7B_7',
#   ============================  여행 인원수
    'A7',
#   ============================  15세 미만 동만자 수
    'A7A',
#   ============================  여행지 지출 총합
    'N_A10',
#   ============================  전반적 만족도
    'A12',
#   ============================  성별
    'SEX',
#   ============================  나이
    'AGE_1YEAR_UNIT',
#   ============================  출발 월
    'BMON',
#   ============================  거주지
    'BSIDO'
]

In [149]:
# 컬럼명 맞게 들어갔는지 체크
for one in columns_list:
    df[one].head(1)

In [150]:
df.columns

Index(['ID', 'MON_EXP_DOM_TOUR', 'MON_EXP_DOM_BUSIN', 'MON_EXP_DOM_FRI',
       'YEAR_EXP_DOM_TOUR', 'YEAR_EXP_DOM_BUSIN', 'YEAR_EXP_DOM_FRI',
       'D_TRA1_CASE', 'D_TRA1_SYEAR', 'D_TRA1_SMONTH',
       ...
       '국내_여행방문지_기타당일7', '국내_여행방문지_기타당일8', '국내_여행방문지_기타당일9', '국내_여행방문지_기타당일10',
       '국내_여행방문지_기타당일11', '국내_여행방문지_기타당일12', '국내_여행방문지_기타당일13',
       '국내_여행방문지_기타당일14', '국내_여행방문지_기타당일15', '국내_여행방문지_기타당일16'],
      dtype='object', length=1302)

In [152]:
df[columns_list]

,A13,A14,A3_1,A3_2,A3_3,A3_4,A3_5,A3_6,A3_7,A3_8,...,A7B_6,A7B_7,A7,A7A,N_A10,A12,SEX,AGE_1YEAR_UNIT,BMON,BSIDO
3,5.0,4.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,360000.000000,4.0,2,62,7,11
5,4.0,3.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,33.0,0.0,57200.000000,4.0,2,53,5,11
6,4.0,4.0,1.0,2.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,5.0,0.0,90000.000000,4.0,1,76,5,11
9,4.0,3.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,68500.000000,4.0,2,66,5,11
10,4.0,4.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,42003.514938,4.0,2,65,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24114,3.0,3.0,1.0,NaN,3.0,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,3.0,1.0,45283.018868,4.0,2,46,8,39
24115,4.0,2.0,NaN,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,1.0,NaN,318181.818182,4.0,2,54,8,39
24126,4.0,4.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,8.0,...,NaN,NaN,5.0,0.0,30000.000000,4.0,2,16,12,39
24132,4.0,4.0,1.0,2.0,NaN,NaN,5.0,6.0,NaN,NaN,...,NaN,NaN,4.0,2.0,127941.176471,4.0,2,26,9,39


In [155]:
df[df["D_TRA1_1_SYEAR"]!=df['D_TRA1_1_EYEAR']].head()

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
1165,11130700151_1223104,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,11130762371_1201154,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,1114070066A_101092,1,2,2,NaN,NaN,NaN,2.0,2017.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,11240530281_1202386,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2877,21010540601_1202813,1,2,2,1.0,2.0,2.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
df[df["D_TRA1_SMONTH"]!=df['D_TRA1_EMONTH']]

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
1165,11130700151_1223104,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1176,1113071084A_703602,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,11130762371_1201154,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,1114070066A_101092,1,2,2,NaN,NaN,NaN,2.0,2017.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,11240530281_1202386,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23463,39010120311_724855,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23488,39010121281_1222623,1,2,2,1.0,2.0,2.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23491,39010121361_747628,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23794,3901060089A_748008,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# 여행 박 수 계산을 위한 라이브러리 import
from datetime import datetime

In [208]:
now  = datetime.now()
date_to_compare = datetime.strptime("2022512", "%Y%m%d")
date_diff = now - date_to_compare
print("차이 :", date_diff.days)
type(date_diff.days)

차이 : 0


int

In [220]:
df.loc[6,'D_TRA1_SYEAR']

2018.0

In [216]:
df['D_TRA1_SYEAR'].fillna(2018, inplace=True)
df['D_TRA2_SYEAR'].fillna(2018, inplace=True)
df['D_TRA3_SYEAR'].fillna(2018, inplace=True)
df['D_TRA4_SYEAR'].fillna(2018, inplace=True)
df['D_TRA1_EYEAR'].fillna(2018, inplace=True)
df['D_TRA2_EYEAR'].fillna(2018, inplace=True)
df['D_TRA3_EYEAR'].fillna(2018, inplace=True)
df['D_TRA4_EYEAR'].fillna(2018, inplace=True)

In [225]:
df['D_TRA1_1_SYEAR'].isna().sum()

23

In [ ]:
for i in range(1, len(df)-1):
    

In [ ]:
start=str(df['D_TRA1_SYEAR'])+str(df['D_TRA1_SMONTH'])+str(df['D_TRA1_SDAY'])
end=str(df['D_TRA1_EYEAR'])+str(df['D_TRA1_EMONTH'])+str(df['D_TRA1_EDAY'])

In [ ]:
## 퍼진 컬럼들 압축 과정

## TRA1_1, TRA1_2 등 모든 여행지 파생변수 area로 ,로 구분해서 입력, 
## 여행지 개수 count하여 no_of_place 컬럼에입력
# 1-1 1-2 1-3 1-4 1-5 ... 8-10
# [1,2,3,4,5,6]
sleepnights_list = [] #여행지에서 잔 밤수(박) ex) 1박 2일 = 1
cost_list = [] # 여행비용 <=이후 중복컬럼 생겨서 미사용
area_list = [] # 거주지 
count_list = [] # 여행지 개수 일치확인용 
month_list = [] # 여행 출발 월
accommodation_list = []
for index in df.index:
    sleepnights = ''
    cost = ''
    area = ''
    count = 0
    month = ''
    accommodation = ''
    for i in range (1,5):
        col_sleepnights = 'D_TRA'+str(i)+'_S_Day'              
        try: 
            sleepnights = sleepnights + str(int(df.loc[index,col_sleepnights])) + ','
        except: pass
        for j in range(1,11):

            col_area ='D_TRA'+str(i)+'_'+str(j)+'_SPOT'
            col_month = 'D_TRA'+str(i)+'_'+str(j)+'_SMONTH'
            col_accommodation = 'D_TRA'+str(i)+'_'+str(j)+'_Q6'
            try: 
                area = area + str(int(df.loc[index, col_area])) + ','
                count += 1
            except: pass
            try:
                month = month + str(int(df.loc[index, col_month])) + ','
            except:
                pass
            try:
                accommodation = accommodation + str(int(df.loc[index, col_accommodation])) + ','
            except:
                pass
    try:
        area = area.rstrip(area[-1])
    except:pass
    try:
        sleepnights = sleepnights.rstrip(sleepnights[-1])
    except:pass
    try:
        cost = cost.rstrip(cost[-1])
    except:pass
    try:
        month = month.rstrip(month[-1])
    except:pass
    try:
        accommodation = accommodation.rstrip(accommodation[-1])
    except:pass
    area_list.append(area)
    count_list.append(count)
    month_list.append(month)
    cost_list.append(cost)
    accommodation_list.append(accommodation)
    sleepnights_list.append(sleepnights)

df['area']= area_list
df['no_of_place'] = count_list
df['month'] = month_list
df['cost'] = cost_list
df['sleepnights'] = sleepnights_list
df['accommodation'] = accommodation_list